In [1]:
import torch
import torchvision.models as models
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import csv
from util import *
%load_ext autoreload
%autoreload 2

In [ ]:
model_name = "SinglepathNAS"
dataflow = "rs"

input_size = (3, 224, 224)
num_classes = 1000
flops = 0
profiled_layers = []
blocks_args = [
      'r1_k3_s22_e3.0_i32_o16', 
      'r1_k3_s11_e3.0_i16_o24',
      'r1_k3_s11_e3.0_i24_o24', 
      'r1_k5_s22_e6.0_i24_o24',
      'r1_k3_s11_e3.0_i24_o40',
      'r1_k3_s11_e3.0_i40_o40',
      'r1_k3_s11_e3.0_i40_o40', 
      'r1_k5_s22_e6.0_i40_o80',
      'r1_k3_s11_e3.0_i80_o80', 
      'r1_k3_s11_e3.0_i80_o80',
      'r1_k3_s11_e3.0_i80_o80',
      'r1_k5_s11_e6.0_i80_o96', 
      'r1_k5_s11_e3.0_i96_o96',
      'r1_k5_s11_e3.0_i96_o96', 
      'r1_k5_s11_e3.0_i96_o96',
      'r1_k5_s22_e6.0_i96_o192',
      'r1_k5_s11_e6.0_i192_o192',
      'r1_k5_s11_e6.0_i192_o192', 
      'r1_k5_s11_e6.0_i192_o192',
      'r1_k3_s11_e6.0_i192_o320',
]

# Stem part
layer_type, kernel_size, stride, out_channels = "Conv", 3, (2, 2), 32
output_size, nb_params, R, S, flop = \
    get_conv_output_and_params_and_flops(input_size, layer_type, kernel_size, stride, out_channels=out_channels)
profiled_layers.append(tuple((layer_type, input_size, output_size, stride, nb_params, R, S)))
flops += flop

# SinglepathNAS: None

# Backbone part
for blocks_string in blocks_args:
    layers, output_size, flop = profile_blockargs(blocks_string, output_size)
    profiled_layers += layers
    flops += flop

# Head part
layer_type, input_size, kernel_size, stride, out_channels = "Conv", output_size, 1, (1, 1), 1280
output_size, nb_params, R, S, flop = \
    get_conv_output_and_params_and_flops(input_size, layer_type, kernel_size, stride, out_channels=out_channels)
profiled_layers += [tuple((layer_type, input_size, output_size, stride, nb_params, R, S))]
flops += flop

layer_type, input_size, out_features = "Linear", output_size, num_classes
output_size, nb_params, R, S, flop = get_linear_output_size_and_nb_param(input_size, out_features, use_pool=True)
profiled_layers += [tuple((layer_type, input_size, output_size, None, nb_params, R, S))]
flops += flop

print("Total number of flops:", flops)

In [ ]:
summary = make_summary(profiled_layers, dataflow=dataflow, model_name=model_name)
# Get number of parameters
layer_names = list(summary.keys())
params = list(map(lambda x: int(summary[x]['nb_params']), layer_names))
print("Total number of parameters:", sum(params))

In [ ]:
with open("SinglepathNAS.csv", mode='w') as model_file:
    model_writer = csv.writer(model_file, delimiter=',')
    for blocks_string in blocks_args:
        model_writer.writerow([blocks_string])

In [ ]:
# blocks_args = []
# with open("MnasNet-A1.csv", mode='r') as model_file:
#     model_reader = csv.reader(model_file, delimiter=',')
#     for row in model_reader:
#         blocks_args += row